In [2]:
# Import packages
import poreana as pa
import porems as pms
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import ipywidgets as widgets
import MDAnalysis as mda
import nglview as nv
from IPython.display import display, Markdown, Latex


In [54]:
# Set strings to the files in data set
def set_strings(molecule):

    strings = {}
    # Strcuture for molecule
    if molecule == "water":
        strings["gro_mol"] = "Molecules/water/tip4p2005.gro"
    elif molecule != "water":
        strings["gro_mol"] = "Molecules/{}/{}.gro".format(molecule,molecule)

    # Strings for pure rectangular systems
    strings["dens_box"] = "Pure/{}/box_rectangular_{}_density.h5".format(molecule,molecule)
    strings["mc_box"] = "Pure/{}/box_rectangular_{}_diffusion_smoluchowski.h5".format(molecule,molecule)
    
    # Strings for pore system
    strings["dens"] = "Pore/{}/pore_ideal_{}_density.h5".format(molecule,molecule)
    strings["bin"] = "Pore/{}/pore_ideal_{}_diffusion_einstein.h5".format(molecule,molecule)
    strings["mc"] = "Pore/{}/pore_ideal_{}_diffusion_smoluchowski.h5".format(molecule,molecule)

    # Strings for amoprh pore systems (if exisit in dataset)
    if molecule in ["cyclopentane","hexane","heptane"]:
        strings["dens_a"] = "Pore/{}/pore_amorph_{}_density.h5".format(molecule,molecule)
        strings["bin_a"] = "Pore/{}/pore_amorph_{}_diffusion_einstein.h5".format(molecule,molecule)
        strings["mc_a"] = "Pore/{}/pore_amorph_{}_diffusion_smoluchowski.h5".format(molecule,molecule)
    
    return strings

In [4]:
# Display molcule and system
def display_molecule(strings):
    # Set markdown header
    display(Markdown('# Molecule'))

    # Load and display gro file of the molecule and
    mol = mda.Universe(strings["gro_mol"])
    view = nv.show_mdanalysis(mol)
    display(view)


In [111]:
# Plot density function
def plot_density(strings, amorph):

    # Markdown header
    display(Markdown('# Density'))

    # Plot settings
    plt.figure(figsize=(14,4))
    plt.subplot(1,2,1)
    plt.ylabel("Density (molecules $\mathrm{nm}^{-1}$)")
    plt.xlabel("Reservoir length (nm)")

    plt.xlim([0,10])
    plt.title("Reservoir density")

    # Plot reservoir density
    dens = pa.density.bins(strings["dens"], is_print=False)
    plt.plot(dens["sample"]["data"]["ex_width"],[den for den in dens["num_dens"]["ex"]], label = "Reservoir Density")
    # Check if results for amorph pore exists 
    if amorph and ("dens_a" in strings):
        dens = pa.density.bins(strings["dens_a"], is_print=False)
        plt.plot(dens["sample"]["data"]["ex_width"],[den for den in dens["num_dens"]["ex"]],linestyle="--" ,label = "Reservoir Density (Amorph)")
    plt.plot(dens["sample"]["data"]["ex_width"],[dens["mean"]["ex"] for i in dens["num_dens"]["ex"]], linestyle = "--", label = "Mean Reservoir Density")
    plt.legend()
    # Plot settings
    plt.subplot(1,2,2)
    plt.title("Radial Pore Density")
    plt.ylabel("Density (molecules $\mathrm{nm}^{-1}$)")
    plt.xlabel("Distance from pore center (nm)")
    plt.xlim([0,2.5])

    # Plot radial pore density
    dens = pa.density.bins(strings["dens"], is_print=False)
    plt.plot(dens["sample"]["data"]["in_width"][:-1],[den for den in dens["num_dens"]["in"]], label = "Pore Density")
    plt.plot(dens["sample"]["data"]["ex_width"][:-1],[dens["mean"]["ex"] for i in dens["num_dens"]["ex"][:-1]], linestyle = "--", label = "Mean Reservoir Density")
    plt.axvspan(xmin=2.1, xmax=2.5, facecolor="grey", alpha=0.3, label= "Silanol O")
    plt.legend()
    plt.tight_layout()
    plt.show()

In [60]:
# Diffusion (MC)
def plot_mc_diffusion(strings, amorph):
    plt.figure(figsize=(14,4))
    # Plot bin diffusion
    plt.subplot(1,2,1)
    plt.title("Bin diffusion")
    mc_box = pa.diffusion.mc_fit(strings["mc_box"], len_step = [10,20,30,40,50,60], is_plot=False, is_print=False)
    diff_bin = pa.diffusion.bins(strings["bin"])
    ax  =sns.lineplot(x=np.linspace(0,2.51,100),y=[mc_box[0] for i in range(100)], linestyle="--", label="Box Diffusion")
    pa.diffusion.bins_plot(diff_bin, kwargs = {"label": "Bin diffusion"})
    if amorph and ("dens_a" in strings):
        diff_bin = pa.diffusion.bins(strings["bin_a"])
        pa.diffusion.bins_plot(diff_bin, kwargs = {"linestyle": "--","label": "Bin diffusion (Amorph)"})
    plt.axvspan(xmin=2.1, xmax=2.5, facecolor="grey", alpha=0.3, label = "Silanol O")
    plt.xlim([0,2.5])
    ax.set_ylim(bottom=0)
    plt.legend(loc=1)
    plt.text(0.05, 0.08, 'NVT : D =' + "%.2f" % (mc_box[1]) +"$\\cdot 10^{-9}$ \nPore : D =" + "%.2f" % (0) +"$\\cdot 10^{-9}$"  ,  bbox={'facecolor': 'white', 'alpha': 0.8, 'pad': 4})
    

    plt.subplot(1,2,2)
    display(Markdown('# Diffusion Profile'))
    plt.title("MC Diffusion")
    mc_box = pa.diffusion.mc_fit(strings["mc_box"], len_step = [10,20,30,40,50,60], is_plot=False, is_print=False)
    mc_pore = pa.diffusion.mc_profile(strings["mc"], len_step = [10,20,30,40,50,60], kwargs={"label": "Pore System"})
    if amorph and ("dens_a" in strings):
            mc_pore = pa.diffusion.mc_profile(strings["mc_a"], len_step = [10,20,30,40,50,60], kwargs={"linestyle":"--","label": "Pore System (Amorph)"})
    pore = pa.diffusion.mc_fit(strings["mc"], len_step = [10,20,30,40,50,60], section = [15,19.7], is_plot=False, is_print=False)
    res = pa.diffusion.mc_fit(strings["mc"], len_step = [10,20,30,40,50,60], section = [0,7], is_plot=False, is_print=False)
    sns.lineplot(x=np.linspace(0,30.1,100),y=[mc_box[0] for i in range(100)], linestyle="--", label="Box Diffusion")
    plt.text(21.7, min(mc_pore[0]), 'NVT : D = ' + "%.2f" % (mc_box[1]) + "$\\cdot 10^{-9}\ \\frac{m^2}{s}$ \nRes  : D = " + "%.2f" % (res[0]) + "$\\cdot 10^{-9}\ \\frac{m^2}{s}$ \nPore : D = " + "%.2f" % (pore[0]) +"$\\cdot 10^{-9}\ \\frac{m^2}{s}$"  ,  bbox={'facecolor': 'white', 'alpha': 0.8, 'pad': 4})

    # Plot area
    plt.axvspan(xmin=0, xmax=10, facecolor="grey", alpha=0.3, label = "Reservoir")
    plt.axvspan(xmin=20, xmax=31, facecolor="grey", alpha=0.3)
    plt.axvspan(xmin=15, xmax=19.7, facecolor="red", alpha=0.3, label = "Evaluated")
    plt.axvspan(xmin=23, xmax=31, facecolor="red", alpha=0.3)
    plt.legend()
    plt.tight_layout()
    plt.show()

In [146]:
# Graphic user interface
def view_results(molecule,amorph):
    strings = set_strings(molecule)
    if type(molecule) != list: 
        display_molecule(strings)
    plot_density(strings, amorph)
    plot_mc_diffusion(strings, amorph)

def update_drop(groups, check):
    dict = {}
    dict["alcohol"] = ["methanol","ethanol","1-propanol","1-butanol",]
    dict["aromatic"] = ["benzene","toluene","pyrrole","pyridine"]
    dict["alkene"] =   ["hexane","heptane","cyclopentane","cylcohexane",]
    dict["others"] =   ["water","tetrahydrofuran"]
    if check==False:
        drop = widgets.Dropdown(options=dict[groups]
        )   
        check_a = widgets.Checkbox(
        value=False,
        description='Display amorph results',
        )
        widgets.interact(view_results, molecule = drop, amorph = check_a)
    else:
        drop = dict[groups]
        check_a = False
        view_results(drop,check_a)



def dashboard():
    display(Markdown("# Overview   \n  The data set contains the following systems at a Temperature $T=295 K$: \n"))
    display(Markdown("<div style=\"margin-left: auto;         margin-right: auto;            width: 30%\"> \n\n |Alcohol   |Aromatic|Alkene      |\n"    "|:-------------|:---------|:-----------|\n"    "|Methanol  |Benzene |Cyclopentane|\n"    "|Ethanol   |Toluene |Cyclohexane |\n"    "|1-Propanol  |Pyrole  |Hexane      |\n"    "|1-Butanol |Pyridine|Heptane     |\n</div>"))

    display(Markdown("# Set molecule"))

    drop_group = widgets.RadioButtons(options=["alcohol","aromatic", "alkene", "others"]
    )

    check_group = widgets.Checkbox(
    value=False,
    description='Display group',
    )

    widgets.interact(update_drop, groups = drop_group, check=check_group)
    



In [147]:
# Display dashboard
dashboard()

# Overview   
  The data set contains the following systems at a Temperature $T=295 K$: 


<div style="margin-left: auto;         margin-right: auto;            width: 30%"> 

 |Alcohol   |Aromatic|Alkene      |
|:-------------|:---------|:-----------|
|Methanol  |Benzene |Cyclopentane|
|Ethanol   |Toluene |Cyclohexane |
|1-Propanol  |Pyrole  |Hexane      |
|1-Butanol |Pyridine|Heptane     |
</div>

# Set molecule

interactive(children=(RadioButtons(description='groups', options=('alcohol', 'aromatic', 'alkene', 'others'), …